In [ ]:
# Variables for Initialization of the data frame for Index 4 - Chapters.csv
file_names = []
found_file_names = []
chapters = []
levels = []
pages = []
all_df_chapters = pd.DataFrame({'file_names': file_names, 'found_file_names': found_file_names, 'chapters': chapters, 'pages' : pages, 'levels' : levels})

for current_chunk in range(1, chunk+1):
    current_df_outlines = df_outlines[df_outlines.chunk_nos == current_chunk]
    
    chunk_has_TOC = 0
    chunk_noOutline_hasTOC = 0
    
    file_name_containing_index = '0'
    
    p_file_names = []
    p_found_file_names = []
    p_chapters = []
    p_levels = []
    p_pages = []
    level = 0
    # Going through all the PDFs - Chunk by Chunk
    for i in range(len(current_df_outlines)):
        # First file in the chunk has Outline 
        if current_df_outlines.iloc[i]['TOC_present'] == 1:
            chunk_has_TOC = 1
            print("stage 1")
            pdfread = p2.PdfFileReader(current_df_outlines.iloc[i]['file_paths'])
            s = pdfread.outlines
            
            nested_is_instance_check(s, 
                                     level, 
                                     current_df_outlines.iloc[i]['file_names'], 
                                     p_file_names, p_found_file_names, 
                                     p_chapters, p_levels, p_pages)
            file_name_containing_index = current_df_outlines.iloc[i]['file_names']
            continue
            
        if count_occorances(p_pages, -999) > 1 and chunk_has_outline == 1:
            print("stage 2: file is {}".format(current_df_outlines.iloc[i]['file_names']))
            start_flag = 0
            page_no = -999
            #print(p_pages)
            for j in range(len(p_chapters)):
                if p_pages[j] >= 0 and p_levels[j] < 5: # probably could have also used p_pages[j] >= - 998
                    print("start flag is on")
                    start_flag = 1
                    continue
                
                if start_flag != 1:
                    continue
                    
                if start_flag == 1 and p_pages[j] <= 0 and p_levels[j] < 5:
                    print("stage 3: finding page no for {} with page{}".format(p_chapters[j], p_pages[j]))
                    page_no, found_file_name = find_page_no(current_df_outlines.iloc[i]['file_paths'], 
                                                            p_chapters[j])
                    p_pages[j] = page_no
                    p_found_file_names = current_df_outlines.iloc[i]['file_names']
                    
                if page_no == -999 and p_levels[j] < 5:
                    print("Not found")
                    break # if the first keyword is not found then dont look any further
                else:
                    print("New Found ")
            
    df_chapters = pd.DataFrame({'file_names': p_file_names, 'found_file_names': p_found_file_names, 'chapters': p_chapters, 'pages' : p_pages, 'levels' : p_levels})
    all_df_chapters = pd.concat([all_df_chapters, df_chapters])
        
    
    